In [1]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import MySQLdb
import datetime
from sklearn.metrics import auc, mean_squared_error

In [2]:
import codecs
import lxml
from lxml import etree
import os
from mmap import mmap
import re
from os import listdir
from os.path import isfile, join
import MySQLdb
from datetime import datetime

In [3]:
import matplotlib as mpl
import datetime

## Запишем в базу

In [4]:
whole_res = pd.read_csv('../data/4ypred.csv', sep=';')

In [5]:
def proccess_flight(r):
    cur.execute("INSERT INTO Forecast(fltNr, opsSuffix, Code, SegOrig, SegDest, TicketActual, TicketForecast, DptDate, reportDt) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                (int(r[1][whole_res.columns.get_loc("fltNr")]), r[1][whole_res.columns.get_loc("opsSuffix")], r[1][whole_res.columns.get_loc("Code")],
                 r[1][whole_res.columns.get_loc("SegOrig")], r[1][whole_res.columns.get_loc("SegDest")], r[1][whole_res.columns.get_loc("actual")], 
                 r[1][whole_res.columns.get_loc("predictions")], r[1][whole_res.columns.get_loc("fltNrDptDt")], ts))

In [6]:
whole_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459216 entries, 0 to 459215
Data columns (total 24 columns):
Unnamed: 0        459216 non-null int64
FlightID          459216 non-null int64
Code              459216 non-null int64
resHoldTotal      459216 non-null float64
DptDt             459216 non-null object
fltNrDptDt        459216 non-null object
fltNrArrDt        459216 non-null object
DptTime           459216 non-null int64
ArrTime           459216 non-null int64
flightNrSf        459216 non-null int64
segment           459216 non-null int64
DayOfWeekDpt      459216 non-null int64
WeekOfMonthDpt    459216 non-null int64
MonthDpt          459216 non-null int64
QuarterDpt        459216 non-null int64
YearDpt           459216 non-null int64
IsWeekendDpt      459216 non-null int64
DayOfWeekArr      459216 non-null int64
WeekOfMonthArr    459216 non-null int64
MonthArr          459216 non-null int64
QuarterArr        459216 non-null int64
YearArr           459216 non-null int64
IsWe

In [7]:
# fltNr
flightNrSf_dict = {0: '10J', 1: '50M', 2: '9J', 3: '49J', 4: '337J', 5: '337ZH', 6: '138J', 7: '137YQ', 8: '102J', 9: '101J', 10: '88J', 11: '87J'}
whole_res['flightNrSf'] = whole_res['flightNrSf'].map(lambda x: flightNrSf_dict.get(x))
def map_x(x):
    if x == '337ZH' or x == '137YQ':
        return x[:-2]
    else:
        return x[:-1]
        
whole_res['fltNr'] = whole_res['flightNrSf'].map(lambda x: map_x(x))
def map_x(x):
    if x == '337ZH' or x == '137YQ':
        return x[-2:]
    else:
        return x[-1:]

whole_res['opsSuffix'] = whole_res['flightNrSf'].map(lambda x: map_x(x))

# seg
segment_dict = {0: '32_8', 1: '32_60', 2: '32_33', 3: '32_58', 4: '8_60', 5: '8_33', 6: '8_58', 7: '60_33', 8: '60_58', 9: '33_58', 10: '32_26', 11: '32_0', 12: '32_29', 13: '32_61', 14: '32_23', 15: '32_45', 16: '8_26', 17: '8_0', 18: '8_29', 19: '8_61', 20: '8_23', 21: '8_45', 22: '26_0', 23: '26_29', 24: '26_60', 25: '26_61', 26: '26_23', 27: '26_33', 28: '26_45', 29: '26_58', 30: '0_29', 31: '0_60', 32: '0_61', 33: '0_23', 34: '0_33', 35: '0_45', 36: '0_58', 37: '58_60', 38: '58_8', 39: '58_32', 40: '60_8', 41: '60_32', 42: '8_32', 43: '58_31', 44: '58_29', 45: '58_0', 46: '58_26', 47: '31_60', 48: '31_29', 49: '31_0', 50: '31_26', 51: '31_8', 52: '31_32', 53: '60_29', 54: '60_0', 55: '60_26', 56: '29_0', 57: '29_26', 58: '29_8', 59: '29_32', 60: '0_26', 61: '0_8', 62: '0_32', 63: '26_8', 64: '26_32', 65: '58_33', 66: '58_61', 67: '58_41', 68: '58_17', 69: '58_24', 70: '58_42', 71: '58_48', 72: '58_56', 73: '58_19', 74: '58_7', 75: '58_25', 76: '58_20', 77: '58_6', 78: '58_35', 79: '58_37', 80: '58_44', 81: '58_51', 82: '33_61', 83: '33_60', 84: '33_41', 85: '33_17', 86: '33_24', 87: '33_42', 88: '33_48', 89: '33_56', 90: '33_19', 91: '33_7', 92: '33_25', 93: '33_20', 94: '33_6', 95: '33_35', 96: '33_37', 97: '33_44', 98: '33_51', 99: '61_60', 100: '61_41', 101: '61_17', 102: '51_44', 103: '51_35', 104: '51_6', 105: '51_20', 106: '51_25', 107: '51_7', 108: '51_19', 109: '51_56', 110: '51_48', 111: '51_42', 112: '51_24', 113: '51_17', 114: '51_41', 115: '51_60', 116: '51_61', 117: '51_31', 118: '51_33', 119: '51_58', 120: '44_35', 121: '44_6', 122: '44_20', 123: '44_25', 124: '44_7', 125: '44_19', 126: '44_56', 127: '44_48', 128: '44_42', 129: '32_5', 130: '32_27', 131: '32_21', 132: '32_9', 133: '32_59', 134: '32_39', 135: '32_13', 136: '5_26', 137: '5_27', 138: '5_29', 139: '5_21', 140: '5_60', 141: '5_9', 142: '5_59', 143: '5_61', 144: '5_39', 145: '5_33', 146: '5_13', 147: '5_45', 148: '5_58', 149: '26_27', 150: '26_21', 151: '26_9', 152: '26_59', 153: '26_39', 154: '26_13', 155: '27_29', 156: '27_21', 157: '27_60', 158: '27_9', 159: '29_60', 160: '29_61', 161: '32_14', 162: '58_9', 163: '29_23', 164: '29_33', 165: '29_45', 166: '29_58', 167: '60_61', 168: '60_23', 169: '60_45', 170: '61_23', 171: '61_33', 172: '61_45', 173: '61_58', 174: '23_33', 175: '23_45', 176: '23_58', 177: '33_45', 178: '45_58', 179: '58_5', 180: '33_9', 181: '33_29', 182: '33_5', 183: '33_8', 184: '33_32', 185: '9_60', 186: '9_29', 187: '9_5', 188: '9_8', 189: '9_32', 190: '60_5', 191: '29_5', 192: '5_8', 193: '5_32', 194: '44_24', 195: '44_17', 196: '5_0', 197: '5_14', 198: '44_41', 199: '32_54', 200: '5_54', 201: '44_60', 202: '44_61', 203: '44_31', 204: '44_33', 205: '44_58', 206: '35_6', 207: '35_20', 208: '35_25', 209: '35_7', 210: '26_14', 211: '51_22', 212: '35_22', 213: '35_56', 214: '35_48', 215: '35_42', 216: '35_24', 217: '35_41', 218: '35_60', 219: '35_61', 220: '35_33', 221: '35_58', 222: '25_7', 223: '50_33', 224: '50_58', 225: '50_46', 226: '50_10', 227: '50_4', 228: '50_18', 229: '50_43', 230: '50_15', 231: '50_62', 232: '50_36', 233: '50_3', 234: '50_55', 235: '33_46', 236: '33_10', 237: '33_4', 238: '33_18', 239: '33_43', 240: '33_15', 241: '33_62', 242: '33_36', 243: '33_3', 244: '33_55', 245: '58_46', 246: '58_10', 247: '58_4', 248: '58_18', 249: '58_43', 250: '58_15', 251: '58_62', 252: '58_36', 253: '58_3', 254: '58_55', 255: '46_10', 256: '46_4', 257: '46_18', 258: '46_43', 259: '46_15', 260: '46_62', 261: '46_36', 262: '46_3', 263: '46_55', 264: '10_4', 265: '10_18', 266: '10_43', 267: '10_15', 268: '10_62', 269: '10_36', 270: '10_3', 271: '10_55', 272: '4_18', 273: '4_43', 274: '4_15', 275: '4_62', 276: '4_36', 277: '4_3', 278: '4_55', 279: '60_41', 280: '60_24', 281: '60_42', 282: '60_48', 283: '60_56', 284: '60_19', 285: '60_7', 286: '60_25', 287: '60_35', 288: '60_37', 289: '60_44', 290: '60_51', 291: '41_24', 292: '41_42', 293: '41_48', 294: '41_56', 295: '41_19', 296: '41_7', 297: '0_27', 298: '27_61', 299: '27_33', 300: '27_58', 301: '25_56', 302: '25_48', 303: '25_42', 304: '25_24', 305: '25_41', 306: '25_60', 307: '25_61', 308: '25_33', 309: '25_58', 310: '7_56', 311: '7_48', 312: '7_42', 313: '7_24', 314: '7_41', 315: '7_60', 316: '7_61', 317: '61_24', 318: '61_42', 319: '61_48', 320: '61_56', 321: '61_7', 322: '55_3', 323: '55_36', 324: '55_62', 325: '55_15', 326: '55_43', 327: '55_4', 328: '55_10', 329: '55_46', 330: '55_58', 331: '55_33', 332: '55_38', 333: '55_50', 334: '3_36', 335: '7_33', 336: '7_58', 337: '56_48', 338: '56_42', 339: '56_24', 340: '56_41', 341: '56_60', 342: '56_61', 343: '61_25', 344: '61_35', 345: '61_37', 346: '61_51', 347: '55_18', 348: '3_62', 349: '3_15', 350: '3_43', 351: '3_18', 352: '3_4', 353: '3_10', 354: '3_46', 355: '3_58', 356: '3_33', 357: '3_38', 358: '3_50', 359: '36_62', 360: '36_15', 361: '36_43', 362: '36_18', 363: '36_4', 364: '36_10', 365: '36_46', 366: '36_58', 367: '36_33', 368: '36_38', 369: '36_50', 370: '62_15', 371: '62_43', 372: '62_18', 373: '62_4', 374: '62_10', 375: '62_46', 376: '62_58', 377: '62_33', 378: '62_38', 379: '62_50', 380: '15_43', 381: '15_18', 382: '15_4', 383: '15_10', 384: '15_46', 385: '15_58', 386: '15_33', 387: '15_38', 388: '15_50', 389: '43_18', 390: '43_4', 391: '43_10', 392: '43_46', 393: '25_22', 394: '18_43', 395: '18_15', 396: '18_62', 397: '18_36', 398: '18_3', 399: '18_55', 400: '43_15', 401: '43_62', 402: '43_36', 403: '43_3', 404: '43_55', 405: '15_62', 406: '15_36', 407: '15_3', 408: '15_55', 409: '62_36', 410: '62_3', 411: '62_55', 412: '36_3', 413: '36_55', 414: '3_55', 415: '58_12', 416: '46_12', 417: '10_12', 418: '4_12', 419: '43_12', 420: '27_23', 421: '61_29', 422: '61_0', 423: '61_26', 424: '61_8', 425: '61_32', 426: '35_19', 427: '25_19', 428: '43_58', 429: '18_4', 430: '18_10', 431: '18_46', 432: '18_58', 433: '4_10', 434: '4_46', 435: '4_58', 436: '10_46', 437: '10_58', 438: '43_33', 439: '7_22', 440: '46_58', 441: '56_33', 442: '56_58', 443: '48_42', 444: '48_24', 445: '48_41', 446: '43_38', 447: '35_17', 448: '35_31', 449: '6_20', 450: '6_25', 451: '6_7', 452: '6_19', 453: '6_56', 454: '6_48', 455: '43_50', 456: '32_30', 457: '8_9', 458: '8_30', 459: '26_30', 460: '60_9', 461: '60_30', 462: '9_33', 463: '9_30', 464: '9_58', 465: '33_30', 466: '30_58', 467: '58_14', 468: '31_14', 469: '60_14', 470: '29_14', 471: '14_26', 472: '14_8', 473: '14_32', 474: '8_5', 475: '32_16', 476: '32_52', 477: '32_53', 478: '32_49', 479: '16_52', 480: '16_53', 481: '16_49', 482: '52_53', 483: '52_49', 484: '53_49', 485: '58_1', 486: '58_45', 487: '58_13', 488: '51_37', 489: '51_34', 490: '51_28', 491: '51_47', 492: '51_57', 493: '32_2', 494: '32_11', 495: '32_40', 496: '16_2', 497: '16_11', 498: '16_40', 499: '2_11', 500: '2_52', 501: '2_40', 502: '2_53', 503: '2_49', 504: '11_52', 505: '11_40', 506: '11_53', 507: '11_49', 508: '52_40', 509: '40_53', 510: '40_49'}
segment_dict2 = {0: 'ZUBVP', 1: 'NOVKU', 2: 'MUROM', 3: 'SGC', 4: 'CEK', 5: 'SHILV', 6: 'MAKSA', 7: 'IARGL', 8: 'RZN1', 9: 'GFBMH', 10: 'AZOQM', 11: 'NAVAS', 12: 'LANGP', 13: 'BEZNC', 14: 'POTMA', 15: 'TJM', 16: 'VEKVK', 17: 'UZHOV', 18: 'KMNUR', 19: 'NEREK', 20: 'SONKV', 21: 'WINOP', 22: 'YTNPR', 23: 'BARYS', 24: 'ARZM1', 25: 'RYBP', 26: 'SASOV', 27: 'TORBV', 28: 'NEKOZ', 29: 'KOVYL', 30: 'OBSHA', 31: 'KUZOV', 32: 'MOWKZ', 33: 'SYZR1', 34: 'BEZHK', 35: 'BOLOG', 36: 'PYTJA', 37: 'OKUL', 38: 'KUZNT', 39: 'BAZRN', 40: 'URMAR', 41: 'SKX', 42: 'GOJ', 43: 'SVX', 44: 'MVISH', 45: 'CHAPV', 46: 'SXMBV', 47: 'SHSTH', 48: 'KOVRV', 49: 'KZNP', 50: 'PEZ1', 51: 'LEDGL', 52: 'KANAS', 53: 'ZELDO', 54: 'VOEVD', 55: 'NJC', 56: 'IVANV', 57: 'VOLGA', 58: 'KUF', 59: 'NOCKA', 60: 'RUZVK', 61: 'INZA', 62: 'TOX'}
whole_res['segment'] = whole_res['segment'].map(lambda x: segment_dict.get(x))
whole_res.dropna(inplace = True)
whole_res['SegOrig'] = whole_res['segment'].map(lambda x: x.split('_')[0])
whole_res['SegDest'] = whole_res['segment'].map(lambda x: x.split('_')[1])

whole_res['SegOrig'] = whole_res['SegOrig'].map(lambda x: segment_dict2.get(int(x)))
whole_res['SegDest'] = whole_res['SegDest'].map(lambda x: segment_dict2.get(int(x)))

# code
code_dict = {0: 'F', 1: 'C', 2: 'B', 3: 'Y', 4: 'X', 5: 'K'}
whole_res['Code'] = whole_res['Code'].map(lambda x: code_dict.get(x))

In [8]:
# 
whole_res.rename(columns={'flightNrSf':'FltNr', 'Date':'fltNrDptDt','resHoldTotal':'actual'}, inplace = True)

In [9]:
db = MySQLdb.connect(host="194.247.15.117", 
                     port=7806,
                     user="bar",
                     passwd="PASSWORD",
                     db="Inventory")

ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

for r in whole_res.iterrows():
    db.autocommit(True) 
    cur = db.cursor()
    proccess_flight(r)
    cur.close()

OperationalError: (2003, "Can't connect to MySQL server on '194.247.15.117' (10060)")